In [1]:
from tools import *
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, NNConv
from torch_geometric.data import Data


In [2]:
graph4JSON="graphes_JSON/Graphe(4)_features.json"

edges= extract_mapped_edges_from_json(graph4JSON)
feature_matrix_df = extract_node_features_from_json_file(graph4JSON)
repartition=extract_optimal_repartition_from_json(graph4JSON)


In [3]:
feature_matrix_df.head()

,fan_in,fan_out,depth,cpu_speed,comm_speed,latency,computation_time,n_cpus
node_id,,,,,,,,
4,1,1,2,5.18,6.48,0.952222,0.41463,64
23,1,1,2,5.18,6.48,0.952222,0.41463,64
39,4,2,2,5.18,6.48,0.952222,1.65852,64
42,1,0,3,5.18,6.48,0.952222,0.41463,64
47,1,0,3,5.18,6.48,0.952222,0.41463,64


In [4]:
edges

,from,to,original_from,original_to
0,0,110,4,0
1,1,113,23,17
2,2,3,39,42
3,2,4,39,47
4,5,32,77,65
...,...,...,...,...
142,123,126,218,230
143,123,124,218,222
144,123,129,218,236
145,132,133,33,34


In [5]:
repartition

,original_id,assigned_cpu,mapped_id
0,4,15,0
1,23,16,1
2,39,22,2
3,42,23,3
4,47,4,4
...,...,...,...
130,252,25,130
131,256,6,131
132,33,7,132
133,34,10,133


In [223]:
# edges
from_nodes=edges['from'].values
to_nodes=edges['to'].values
edge_index = np.array([from_nodes, to_nodes], dtype=np.int64)
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)

# x_train
features=feature_matrix_df.values
x = torch.tensor(features, dtype=torch.float32)

# y_train
y_target = repartition['assigned_cpu'].values
y = torch.tensor(y_target, dtype=torch.long)


data = Data(x=x, edge_index=edge_index_tensor, y=y)

In [224]:
data

Data(x=[135, 8], edge_index=[2, 147], y=[135])

In [225]:
class GNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(in_channels=8, out_channels=32)  # 8 node features -> 32 hidden
        self.lin = torch.nn.Linear(32, 64)  # 64 classes (CPUs)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)  # Each node gathers info from 1-hop neighbors
        x = F.relu(x) 
        embeddings = x.clone()                  # Non-linearity
        x = self.lin(x)                # Final layer predicts a class between 0–63
        return x,embeddings

In [226]:
model = GNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

In [227]:
model

GNN(
  (conv1): GCNConv(8, 32)
  (lin): Linear(in_features=32, out_features=64, bias=True)
)

In [ ]:
all_losses = []
all_accuracies = []
all_embeddings=[]

for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    out , embeddings= model(data)
    loss = loss_fn(out, data.y)
    loss.backward()
    optimizer.step()
    
    _, pred = out.max(dim=1)
    correct = (pred == data.y).sum()
    acc = int(correct) / data.y.size(0)
    
    all_losses.append(loss.item())
    all_accuracies.append(acc)
    all_embeddings.append(embeddings)
    
    if epoch % 100 == 9 or epoch == 0:  
        print(f"Epoch {epoch+1:2d}: Loss: {loss.item():.4f}, Accuracy: {acc:.4f}")


Epoch  1: Loss: 14.8860, Accuracy: 0.0148
Epoch 10: Loss: 5.2278, Accuracy: 0.0222
Epoch 110: Loss: 3.5107, Accuracy: 0.0963
Epoch 210: Loss: 3.1022, Accuracy: 0.0815
Epoch 310: Loss: 2.9708, Accuracy: 0.1259
Epoch 410: Loss: 2.8584, Accuracy: 0.1630
Epoch 510: Loss: 2.7133, Accuracy: 0.1852
Epoch 610: Loss: 2.5349, Accuracy: 0.2222
Epoch 710: Loss: 2.4003, Accuracy: 0.2815
Epoch 810: Loss: 2.2914, Accuracy: 0.3185
Epoch 910: Loss: 2.1986, Accuracy: 0.3259
Epoch 1010: Loss: 2.1093, Accuracy: 0.3704
Epoch 1110: Loss: 2.0362, Accuracy: 0.3704
Epoch 1210: Loss: 1.9765, Accuracy: 0.3852
Epoch 1310: Loss: 1.9241, Accuracy: 0.3852
Epoch 1410: Loss: 1.9017, Accuracy: 0.4074
Epoch 1510: Loss: 1.8402, Accuracy: 0.4222
Epoch 1610: Loss: 1.8049, Accuracy: 0.4370
Epoch 1710: Loss: 1.7748, Accuracy: 0.4222
Epoch 1810: Loss: 1.7459, Accuracy: 0.4222
Epoch 1910: Loss: 1.7185, Accuracy: 0.4444
Epoch 2010: Loss: 1.6902, Accuracy: 0.4444
Epoch 2110: Loss: 1.6682, Accuracy: 0.4444
Epoch 2210: Loss: 1.650

In [229]:
y_pred,embeddings= model(data)
predicted_classes = y_pred.argmax(dim=1)



In [230]:
pred_np = predicted_classes.cpu().numpy()
actual_np = data.y.cpu().numpy()

predicitons_df = pd.DataFrame({
    'Actual': actual_np,
    'Predicted': pred_np
})

In [231]:
predicitons_df

,Actual,Predicted
0,15,15
1,16,15
2,22,22
3,23,23
4,4,23
...,...,...
130,25,25
131,6,6
132,7,7
133,10,10
